In [4]:
import pandas as pd
import time
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession, types
import pyspark.sql.functions as F
from  pyspark.sql.functions import col

spark = (
    SparkSession.builder.config("spark.sql.debug.maxToStringFields", 100)
    .appName("reviews")
    .getOrCreate()
)

data_filepath = "../data/cleaned_steam_reviews"
steam_games_filepath = "../data/cleaned_steam_games"

In [5]:
steam_reviews = spark.read.parquet(data_filepath)

In [6]:
steam_games = spark.read.parquet(steam_games_filepath)
steam_games = steam_games.select("game_id", "name", F.explode_outer(F.split(col("popular_tags"),",")).alias("tags")) \
                            .drop("popular_tags") \
                            .where(col("tags") != F.lit("null"))

In [9]:
steam_games.show()

+--------------------+-----+----------------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+--------------------+-------+
|                 url|types|                        name|        desc_snippet|      recent_reviews|         all_reviews|release_date|           developer|           publisher|        popular_tags|        game_details|           languages|achievements|               genre|    game_description|      mature_content|minimum_requirements|recommended_requirements|      original_price|      discount_price|game_id|
+--------------------+-----+----------------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+---

In [11]:
# join app_id and game_id
steam = steam_games.join(steam_reviews, steam_games.game_id == steam_reviews.app_id, "inner")

In [12]:
steam.show()

+--------------------+-----+-------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------+--------------+-------+------+--------+---------+---------+--------------------------------+--------------------+--------------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+-------+
|                 url|types|   name|        desc_snippet|      recent_reviews|         all_reviews|release_date|  developer|           publisher|        popular_tags|        game_details|           languages|achievement

In [13]:
steam.printSchema()

root
 |-- url: string (nullable = true)
 |-- types: string (nullable = true)
 |-- name: string (nullable = true)
 |-- desc_snippet: string (nullable = true)
 |-- recent_reviews: string (nullable = true)
 |-- all_reviews: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- developer: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- popular_tags: string (nullable = true)
 |-- game_details: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- achievements: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- game_description: string (nullable = true)
 |-- mature_content: string (nullable = true)
 |-- minimum_requirements: string (nullable = true)
 |-- recommended_requirements: string (nullable = true)
 |-- original_price: string (nullable = true)
 |-- discount_price: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_name: string (nullable = true)
 |-- rev